In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"


In [2]:
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset, load_from_disk



/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_model = "decapoda-research/llama-7b-hf"
device_map = "auto"

model = LlamaForCausalLM.from_pretrained(
    base_model,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)


Loading checkpoint shards: 100%|██████████| 33/33 [00:10<00:00,  3.08it/s]


In [4]:
tokenizer = LlamaTokenizer.from_pretrained(base_model)
tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token
tokenizer.padding_side = "left"  # Allow batched inference


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [5]:
def generate_and_tokenize_prompt(prompt):
    # print(prompt)
    prompt = prompt["prompt"]

    tokenized_prompt = tokenizer(
        prompt,
        max_length=1024,
        truncation=True,
    )
    user_prompt = prompt[: prompt.index("### Assistant:")]
    tokenized_user_prompt = tokenizer(
        user_prompt,
        max_length=1024,
        truncation=True,
        add_special_tokens=False,
    )
    prompt_len = len(tokenized_user_prompt["input_ids"])

    tokenized_prompt["labels"] = [-100] * (prompt_len) + tokenized_prompt["input_ids"][
        prompt_len:
    ]
    return tokenized_prompt


In [6]:
data_files = {
    "train": "/home/kosenko/ru_chatGPT/ru_instruct/ru_instruct/sandbox/datasets_processed/IlyaGusev_ru_turbo_alpaca_formatted.json"
}
data = load_dataset(
    "json",
    data_files=data_files,
)

# всеравно мы никак не можем адекватно мерить модель
# тогда зачем тратить на валидацию данные?
train_data = (
    data["train"]
    .shuffle()
    .map(
        generate_and_tokenize_prompt,
        num_proc=32,
        # batched=True,
    )
)


Found cached dataset json (/home/kosenko/.cache/huggingface/datasets/json/default-8fca4c8833304c51/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 142.74it/s]


In [7]:
print(data["train"][1000]["prompt"])




A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Как вы относитесь к конкуренции на рабочем месте?

### Response:
### Assistant:
Я считаю, что конкуренция на рабочем месте может быть полезной, если она справедливая и здоровая. Конкуренция может стимулировать улучшение работы и повышение производительности. Однако, я также считаю, что важно сохранять здоровую рабочую среду и отношения с коллегами. Я всегда стараюсь помогать своим коллегам и делиться своими знаниями и опытом, чтобы мы могли достигать успеха вместе.




In [8]:
train_data


Dataset({
    features: ['prompt', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 75368
})

In [9]:
print(train_data[0]["prompt"])
print(tokenizer.decode(train_data[0]["input_ids"]))




A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Напиши краткое объяснение сложной теории или концепции.
Клеточные автоматы.
### Response:
### Assistant:
Клеточный автомат - это математическая модель, представляющая собой сетку клеток, которые могут находиться в одном из конечного числа состояний. Каждая клетка взаимодействует со своими соседями в зависимости от определенных правил, которые могут зависеть от текущего состояния клетки и ее соседей. Такие модели используются для исследования различных явлений, включая распространение эпидемии и физических процессов в материалах, а также для создания сложных компьютерных моделей искусственного интеллекта.


<unk>

A chat between a curious human and an artificial intelligence assistant. The assista

In [10]:
train_data = train_data.remove_columns("prompt")
splited_dataset = train_data.train_test_split(train_size=0.98, test_size=0.02)

In [11]:
splited_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 73860
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1508
    })
})

In [12]:

data_collator = transformers.DataCollatorForSeq2Seq(
	tokenizer=tokenizer,
	pad_to_multiple_of=8,
 	return_tensors="pt", 
  	padding=True
)
dataloader = DataLoader(
	dataset=splited_dataset['train'],
	batch_size=1,
	# num_workers=self.hparams.num_workers,
	pin_memory=True,
	shuffle=True,
 	collate_fn=data_collator
)

In [13]:
example = next(iter(dataloader))
example

{'input_ids': tensor([[    0,     0,     0, 29871,    13,    13, 29909, 13563,  1546,   263,
         12758,  5199,   322,   385, 23116, 21082, 20255, 29889,   450, 20255,
          4076,  8444, 29892, 13173, 29892,   322,  1248,   568,  6089,   304,
           278,  5199, 29915, 29879,  5155, 29889,    13,    13,  2277, 29937,
         12968, 29901, 13866,   338,   385, 15278,   393, 16612,   263,  3414,
         29889, 14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,
          2009, 29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,
         19193,  1668,  1911, 20658,  6195, 10539, 12189,   376,   587, 14370,
          6988,  2031,  2370, 21424,   644, 29972,  1642,    13,    13,  2277,
         29937, 13291, 29901,    13,  2277, 29937,  4007, 22137, 29901,    13,
         30027, 29919, 14370,  6988,  2031,  2370, 21424,   644, 29972,   448,
          6408,  2591, 19851, 14760, 20148, 29957, 20565,  1587, 29892, 12423,
          8838, 29982, 25107,   665,  

In [15]:
output = model(**example)

In [17]:
output.loss

tensor(0.8252, dtype=torch.float16, grad_fn=<ToCopyBackward0>)

In [1]:
list(['/home/kosenko/ru_chatGPT/ru_instruct/ru_instruct/sandbox/datasets_processed/IlyaGusev_ru_turbo_alpaca_formatted.json'])

['/home/kosenko/ru_chatGPT/ru_instruct/ru_instruct/sandbox/datasets_processed/IlyaGusev_ru_turbo_alpaca_formatted.json']